In [ ]:
from lxml import etree as ET
import resource
from tqdm import tqdm

path = []
results = []
to_process = False
root = "{http://www.mediawiki.org/xml/export-0.10/}"


for event, elem in tqdm(ET.iterparse("commonswiki-20221001-pages-articles-multistream.xml", events=("start", "end"))):
    if event == 'end':
        if elem.tag in (root+"ns",root+"text",root+"sha1"):
            if elem.tag == root+"ns":
                if elem.text == "102":
                    print(elem.text)
                    to_process = True

            if to_process == True:
                if elem.tag == root+"text":
                    text = elem.text
                if elem.tag == root+"comment":
                    comment = elem.text
                    results.append({"text":text,"comment":comment})
                    to_process=False
        elem.clear()
            

# Get transcriptions

In [ ]:
# Get pages with NS = 102 (TimedText)
import mwxml
import pickle
from tqdm import tqdm

def get_dataset(xml_file ,output_file_name):
    dump = mwxml.Dump.from_file(open(xml_file))
    print(dump.site_info.name, dump.site_info.dbname)

    pages = []
    counter = 0
    for page in tqdm(dump):
        if page.namespace == 102:
            counter+=1
            for revision in page:
                revision = revision
            pages.append(revision)        
            print(counter)

    wikimedia = [i.to_json() for i in pages]

    import json
    with open(output_file_name , 'w') as file:
        json.dump(wikimedia, file)
    

In [ ]:
import json
with open("wikimedia.json" , 'r') as file:
        previous_json = json.load(file)

In [ ]:
############# ESTE ES EL BUENO ###############

from pyWikiCommons import pyWikiCommons
import requests
import os
from google.cloud import storage
from datetime import timedelta
import os
from tqdm import tqdm


# Esto solo sirve para .webm
def change_format(file_name):
    import subprocess
    changed_filename = os.rename(file_name, "temp.webm")
    subprocess.run(f'ffmpeg -i "temp.webm" {file_name}.mp3',shell=True)
    
def download_file(url,file_name):
    headers = {'User-Agent': 'CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org)'}
    with requests.get(url, stream=True, headers=headers) as r:
        r.raise_for_status()
        with open(file_name, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    return file_name

def split_with_last_dot(string):
    for character in range(len(string)-1,0,-1):
        if string[character]==".":
            left = string[:character]
            right = string[character+1:]
            return [left,right]

def upload_file_gcp(file_name, file_type):    
    # Initiate names
    project_id = 'the-peoples-speech' # Your Project ID
    bucket_name = 'wikimedia_commons' # Name of bucket
    bucket_file = f'{file_type}/{file_name}' # Location of the file on Bucket
    local_file = f'{file_name}' # Location of downloaded file on your PC

    # Set Credential
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "the-peoples-speech-ee8163e65ace.json"
    client = storage.Client(project_id)
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(bucket_file)
    blob.upload_from_filename(local_file)
    signed_url = blob.generate_signed_url(
                    version="v4",
                    expiration=timedelta(seconds=60),
                    method="GET")


    
counter = 0
database = {}
for i in tqdm(range(13746,len(previous_json))):
    link = previous_json[i]["page"]["title"]
    dots = 0
    try:
        if len(link.split("."))>2:
            left,_ = split_with_last_dot(link)
            true_link, _ = split_with_last_dot(left)


            with open(link+".txt", 'w') as f:
                f.write(previous_json[i]["text"])
            upload_file_gcp(link+".txt", "transcription")
            os.remove(link+".txt")

            if true_link in database.keys():
                database[true_link].append(link)
            else:
                database[true_link] = [link]
                try:
                    url = pyWikiCommons.get_commons_url("File:" + true_link)
                    file_name = download_file(url, true_link)
                    upload_file_gcp(true_link, "raw_videos")
                    os.remove(true_link)
                    f = open("uploaded.txt", "a")
                    f.write(f"{i},{true_link}\n")
                    f.close()
                except:
                    f = open("error.txt", "a")
                    f.write(f"{i},{true_link}\n")
                    f.close()
        else:
            print(i)
    except:
        print(i)



In [ ]:
#link = previous_json[2345]["page"]["title"]
#link


In [ ]:
def split_with_last_dot(string):
    for character in range(len(string)-1,0,-1):
        if string[character]==".":
            left = string[:character]
            right = string[character+1:]
            return [left,right]


In [ ]:
from google.cloud import 

In [ ]:
import subprocess

path = 'The Impact of Wikipedia Dumisani Ndubane.webm'

subprocess.run(f'ffmpeg -i {path} perro.mp3',shell=True)

In [ ]:
from gcloud import storage
from oauth2client.service_account import ServiceAccountCredentials
import os

credentials_dict = {
    'type': 'service_account',
    'client_id': os.environ['BACKUP_CLIENT_ID'],
    'client_email': os.environ['BACKUP_CLIENT_EMAIL'],
    'private_key_id': os.environ['BACKUP_PRIVATE_KEY_ID'],
    'private_key': os.environ['BACKUP_PRIVATE_KEY'],
}
credentials = ServiceAccountCredentials.from_json_keyfile_dict(
    credentials_dict
)
client = storage.Client(credentials=credentials, project='myproject')
bucket = client.get_bucket('mybucket')
blob = bucket.blob('myfile')
blob.upload_from_filename('myfile')


In [ ]:
previous_json[0]

In [ ]:
for i in range(60,120):
    print(previous_json[i]["page"]["title"]) 
#'Edit Button.ogv.sl.srt'

In [ ]:
"Wikipedia User Name MEDIUM.ogv.br.srt"[::-1].split('.', 2)[-1][::-1]

In [ ]:
"Wikipedia User Name MEDIUM.ogv.br.srt"[::-1]

In [ ]:
pyWikiCommons.get_commons_url("File:" + true_link)

In [ ]:
len(database.keys())

In [ ]:
database = {}
database.has_key("key")

In [ ]:
import requests

x = requests.get('https://commons.wikimedia.org/wiki/File:Edit_Button.ogv')
print(x.status_code)

In [ ]:
x.json

In [ ]:

    
with open("test", "rb") as fp:
    b = pickle.load(fp)


In [ ]:
len(b)

In [ ]:
b[2]

In [ ]:
url="https://commons.wikimedia.org/wiki/TimedText:Elephants Dream.ogg.sv.srt"

In [ ]:

links = []
for transcription in b:
    comment = transcription["comment"]
    if isinstance(comment, str):
        splitted_comment = comment.split("TimedText:")
        if len(splitted_comment)>1:
            link = []
            for index in range(1,len(splitted_comment)):

                limit = splitted_comment[index].find("]")
                print("links")
                print(splitted_comment[index][:limit])
                link.append(splitted_comment[index][:limit])
            links.append(link)


In [ ]:
!pip3 install pyWikiCommons


In [ ]:
from pyWikiCommons import pyWikiCommons

pyWikiCommons.get_commons_url("File:Elephants Dream.ogg")


In [ ]:
import requests
def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    headers = {'User-Agent': 'CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org)'}
    with requests.get(url, stream=True, headers=headers) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename

In [ ]:
'https://upload.wikimedia.org/wikipedia/commons/d/d5/Elephants_Dream.ogv'

In [ ]:
download_file("https://commons.wikimedia.org/wiki/TimedText:Elephants Dream.ogg.sv.srt&action=raw")

In [ ]:
download_file("https://commons.wikimedia.org/w/index.php?title=TimedText:Elephants_Dream.ogv.en.srt&action=raw")


# Files Database

In [ ]:
import json
with open("database.json" , 'r') as file:
        database = json.load(file)

In [ ]:
with open(r"error.txt", 'r') as fp:
    x = len(fp.readlines())
    print('Total error Videos:', x)

with open(r"uploaded.txt", 'r') as fp:
    x = len(fp.readlines())
    print('Total good Videos:', x)
    

# Change Format

In [ ]:
def change_format(file_name):
    import subprocess
    changed_filename = os.rename(file_name, "temp.webm")
    subprocess.run(f'ffmpeg -i "temp.webm" {file_name}.mp3',shell=True)

change_format("video_1.webm")


# Upload file

In [ ]:
import os
from google.cloud import storage
from datetime import timedelta

# Initiate names
project_id = 'the-peoples-speech' # Your Project ID
bucket_name = 'wikimedia_commons' # Name of bucket
bucket_file = 'test/error.txt' # Location of the file on Bucket
local_file = 'error.txt' # Location of downloaded file on your PC

# Set Credential
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "the-peoples-speech-ee8163e65ace.json"
client = storage.Client(project_id)
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(bucket_file)
blob.upload_from_filename(local_file)
signed_url = blob.generate_signed_url(
                version="v4",
                expiration=timedelta(seconds=60),
                method="GET")
# Print Signed URL
print(signed_url)

In [ ]:
from google.cloud import storage

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

upload_blob("ensayo_1", "error.txt", "error.txt")

In [ ]:
len(new_database.keys())

# Calculate video time

In [ ]:
import json
with open("wikimedia.json" , 'r') as file:
        previous_json = json.load(file)

In [ ]:
import re
hour = 0
minutes = 0
secounds = 0
no_text = []

for register in range(len(previous_json)):
    
    try:
        time = re.findall("(?:([01]?\d|2[0-3]):([0-5]?\d):)?([0-5]?\d)", previous_json[register]["text"])
        for i in reversed(hours):
            if i[0]!="" and i[1]!="" and i[2]!="":
                hour += int(i[0])
                minutes += int(i[1])
                secounds += int(i[2])
                break
    except:
        no_text.append(register)
        

In [ ]:

temp_minutes = minutes + secounds//60
reformated_secounds = secounds%60
reformated_hour = hour + temp_minutes//60
reformated_minuts = minutes % 60
print(f"{reformated_hour}:{reformated_minuts}:{reformated_secounds}")



In [ ]:
from google.cloud import storage

lista = []
client = storage.Client()
for blob in client.list_blobs('wikimedia_commons', prefix='raw_videos'):
    lista.append(blob)

In [ ]:
dir(lista[0])

In [ ]:
lista[500].download_to_file

In [ ]:
languaje count

In [ ]:
def split_with_last_dot(string):
    for character in range(len(string)-1,0,-1):
        if string[character]==".":
            left = string[:character]
            right = string[character+1:]
            return [left,right]

language_count = {}
for key in database.keys():
    for video in database[key]:
        left,_ = split_with_last_dot(video)
        _,language  = split_with_last_dot(left)
        if language not in language_count.keys():
            language_count[language]=1
        else:
            language_count[language]+=1
        

# Change format

In [ ]:
import pathlib
import subprocess
import os
from tqdm import tqdm
from datetime import timedelta

def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    downloads_folder = pathlib.Path.cwd()
    file = downloads_folder.joinpath(destination_file_name)

    blob.download_to_filename(file)

def change_format(file_name):
    subprocess.run(f'ffmpeg -vn -sn -i {file_name} temp.flac',shell=True)
    
def split_with_last_dot(string):
    for character in range(len(string)-1,0,-1):
        if string[character]==".":
            left = string[:character]
            right = string[character+1:]
            return [left,right]
        
def upload_file_gcp(file_name, file_type):    
    # Initiate names
    project_id = 'the-peoples-speech' # Your Project ID
    bucket_name = 'wikimedia_commons' # Name of bucket
    bucket_file = f'{file_type}/{file_name}' # Location of the file on Bucket
    local_file = f'temp.flac' # Location of downloaded file on your PC

    # Set Credential
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "the-peoples-speech-ee8163e65ace.json"
    client = storage.Client(project_id)
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(bucket_file)
    blob.upload_from_filename(local_file)
    signed_url = blob.generate_signed_url(
                    version="v4",
                    expiration=timedelta(seconds=200),
                    method="GET")

with open(r"fail_reformat.txt", 'r') as fp:
    x = fp.readlines()
    
actual_video = 70  
for missing_file in tqdm(x):
    missing_file_converted = f"raw_videos/{missing_file}"[:-6]
    for i in range(70,len(files_list)):
        if files_list[i].name==missing_file_converted:
            file = files_list[i]
            file_name ,ext = split_with_last_dot(file.name)
            actual_video = i
            try:
                download_blob("wikimedia_commons", file.name, f"temp.{ext}")
                if ext != "flac":
                    change_format(f"'temp.{ext}'")
                upload_file_gcp(f"{file_name[11:]}.flac", "reformat")
                if ext != "flac":
                    os.remove(f'temp.{ext}')
                os.remove(f'temp.flac')
                f = open("new_reformat.txt", "a")
                f.write(f"{file_name[11:]}.flac\n")
                f.close()
                print("success")
            except:
                f = open("new_fail_reformat.txt", "a")
                f.write(f"{file.name[11:]}.flac\n")
                f.close()
                print(file.name[11:])
            

Apply ftfy

In [ ]:
############# ESTE ES PARA TRANSCRIPCIONES ###############

import ftfy
from pyWikiCommons import pyWikiCommons
import requests
import os
from google.cloud import storage
from datetime import timedelta
import os
from tqdm import tqdm


# Esto solo sirve para .webm
def change_format(file_name):
    import subprocess
    changed_filename = os.rename(file_name, "temp.webm")
    subprocess.run(f'ffmpeg -i "temp.webm" {file_name}.mp3',shell=True)
    
def download_file(url,file_name):
    headers = {'User-Agent': 'CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org)'}
    with requests.get(url, stream=True, headers=headers) as r:
        r.raise_for_status()
        with open(file_name, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    return file_name

def split_with_last_dot(string):
    for character in range(len(string)-1,0,-1):
        if string[character]==".":
            left = string[:character]
            right = string[character+1:]
            return [left,right]

def upload_file_gcp(file_name, file_type):    
    # Initiate names
    project_id = 'the-peoples-speech' # Your Project ID
    bucket_name = 'wikimedia_commons' # Name of bucket
    bucket_file = f'{file_type}/{file_name}' # Location of the file on Bucket
    local_file = f'{file_name}' # Location of downloaded file on your PC

    # Set Credential
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "the-peoples-speech-ee8163e65ace.json"
    client = storage.Client(project_id)
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(bucket_file)
    blob.upload_from_filename(local_file)
    signed_url = blob.generate_signed_url(
                    version="v4",
                    expiration=timedelta(seconds=60),
                    method="GET")


    
counter = 0
database = {}
for i in tqdm(range(len(previous_json))):
    link = previous_json[i]["page"]["title"]
    dots = 0
    try:
        if len(link.split("."))>2:
            left,_ = split_with_last_dot(link)
            true_link, _ = split_with_last_dot(left)


            with open(link, 'w') as f:
                f.write(ftfy.fix_text(previous_json[i]["text"]))
            upload_file_gcp(link, "transcription")
            os.remove(link)            

            if true_link in database.keys():
                database[true_link].append(link)
            else:
                database[true_link] = [link]
                try:
                    f = open("uploaded.txt", "a")
                    f.write(f"{i},{true_link}\n")
                    f.close()
                except:
                    f = open("error.txt", "a")
                    f.write(f"{i},{true_link}\n")
                    f.close()
        else:
            print(i)
    except:
        print(i)